## Libraires

In [ ]:
# !pip install nltk

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
from sklearn.metrics import accuracy_score
# Download NLTK stopwords (only once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, word_tokenize

# If not done already, download necessary NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import random
import warnings
warnings.filterwarnings("ignore")
nltk.download('wordnet')
from nltk.corpus import wordnet
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from transformers import BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Loading Dataset

In [7]:
dataset_path='/content/drive/MyDrive/FreelanceProjects/StudentSelection/Cleaned_HS_Tech_Internship_Data.xlsx'
df = pd.read_excel(dataset_path)
df

,first_name,last_name,town,identify_as,specify_below,why_internship,tech_experience,non_tech_experience,goals,other_comments,selected
0,Fuad,Ahmed,ST. JOHN'S,Female,South Asian,I am applying for this internship because I be...,I do not have any experience with any technolo...,Quick Learner\nTechnologically competent\nHard...,My plans are to pursue a degree in engineering...,Thank you for considering my application. I am...,no
1,Lucas,Maloney,Avondale,Other,NaN,I am applying for an internship because I am v...,The only experience I have is using computers ...,"I am a hard worker, organized, video gamer, mu...",After graduation in June I will work for the s...,Thank you for considering my application. I am...,no
2,Damon,Pitts,Mount Pearl,Male,White,I am applying for this internship for several ...,I have a variety of experience working with te...,Computer Software / Technical Skills\nâ€¢\tMic...,"After high school graduation, I am considering...",Thank you for considering my application. I am...,no
3,Riley,Hiron,Corner Brook,Male,White,I am applying for this internship in the hope'...,I have some home projects such as making some ...,I know how to program in python and c# and I ...,Post secondary at collage of the north Atlanti...,Thank you for considering my application. I am...,no
4,Keenan,Crawley,Avondale,Male,White,This job opportunity caught my eye. The reason...,"I've built my own computer from scratch, and h...",Google suite for education. Along with the use...,A goal I would want to see fulfilled and in my...,Thank you for considering my application. I am...,no
...,...,...,...,...,...,...,...,...,...,...,...
1832,Chloe,Wilson,Torbay,Female,South Asian,"""I’m eager to learn more about technology and ...","""Have learned basic HTML, CSS, and JavaScript ...","""Volunteered for a community event, learning e...","""I’m considering a career in tech or marketing...",Thank you for considering my application. I am...,yes
1833,Mason,Clark,Gander,Male,White,"""I’m curious about the tech field and unsure i...","""I’ve worked on a small web development projec...","""I’ve volunteered with youth groups, learning ...","""I’m thinking about tech, but I’m also conside...",Thank you for considering my application. I am...,no
1834,Ella,Mitchell,Clarenville,Female,South Asian,"""I’m applying to see if tech could be the righ...","""Created a personal website using WordPress an...","""Worked as a volunteer coordinator for a non-p...","""Exploring tech but also considering public re...",Thank you for considering my application. I am...,no
1835,Ben,Allen,St. John's,Male,White,"""I’m interested in tech but haven’t fully comm...","""Created a portfolio website using HTML, CSS, ...","""Organized local tech meetups for university s...","""Tech exploration with an interest in entrepre...",Thank you for considering my application. I am...,no


In [8]:
df['selected'] = df['selected'].str.strip().str.lower().map({
    'yes': 'yes',
    'no': 'no',
    'declined': 'no'
})

df['selected'].value_counts()

,count
selected,
yes,978
no,859


In [ ]:
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

In [10]:
df = df.dropna(subset=['specify_below', 'tech_experience'])


Series([], dtype: int64)


In [17]:
# Initialize
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words.remove('not')  # Keep 'not' for better negation handling

# POS tag converter
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Clean + lemmatize function
def clean_and_lemmatize(text):
    if pd.isnull(text):
        return ""
    text = str(text)
    text = re.sub(r'\bRT\b', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    text = text.lower()

    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens
        if word not in stop_words
    ]
    return ' '.join(lemmatized_words)

# List of your columns to process
cols_to_clean = ['why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments']

# Apply the cleaning function to each column
for col in cols_to_clean:
    df[col] = df[col].apply(clean_and_lemmatize)


In [18]:
df

,first_name,last_name,town,identify_as,specify_below,why_internship,tech_experience,non_tech_experience,goals,other_comments,selected
0,Fuad,Ahmed,ST. JOHN'S,Female,South Asian,apply internship believe incredible experience...,not experience technology project light progra...,quick learner technologically competent hard w...,plan pursue degree engineering likely software...,thank consider application excite opportunity,no
2,Damon,Pitts,Mount Pearl,Male,White,apply internship several reason firstly intere...,variety experience work technology relate proj...,computer software technical skill microsoft of...,high school graduation consider career enginee...,thank consider application excite opportunity,no
3,Riley,Hiron,Corner Brook,Male,White,apply internship hope improve software develop...,home project make game use c inside unity engi...,know program python c enjoy sharpen skill home...,post secondary collage north atlantic software...,thank consider application excite opportunity,no
4,Keenan,Crawley,Avondale,Male,White,job opportunity catch eye reason enjoy use dif...,build computer scratch download several progra...,google suite education along use steam configu...,goal would want see fulfilled future well educ...,thank consider application excite opportunity,no
5,Seamus,Goodyear,St. Johns,Male,White,grade student already start make decision rega...,not direct experience work specific technology...,technical general computer knowledge microsoft...,highschool graduation plan continue education ...,thank consider application excite opportunity,no
...,...,...,...,...,...,...,...,...,...,...,...
1832,Chloe,Wilson,Torbay,Female,South Asian,eager learn technology explore career opportun...,learn basic html c javascript online tutorial,volunteer community event learn event manageme...,consider career tech marketing still explore o...,thank consider application excite opportunity,yes
1833,Mason,Clark,Gander,Male,White,curious tech field unsure right internship giv...,work small web development project use html ja...,volunteer youth group learn leadership teamwor...,think tech also consider career business sport...,thank consider application excite opportunity,no
1834,Ella,Mitchell,Clarenville,Female,South Asian,apply see tech could right fit still undecided...,create personal website use wordpress basic co...,work volunteer coordinator non profit organiza...,explore tech also consider public relation soc...,thank consider application excite opportunity,no
1835,Ben,Allen,St. John's,Male,White,interested tech fully commit career want learn...,create portfolio website use html c basic java...,organize local tech meetups university student...,tech exploration interest entrepreneurship sma...,thank consider application excite opportunity,no


### BERT Large Model

#### Initial Steps

In [19]:
# 1. Combine all text fields into one
df['combined_text'] = df[['why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments']].agg(' '.join, axis=1)

# 2. Map labels to 0 and 1
df['label'] = df['selected'].map({'yes': 1, 'no': 0})

# 3. Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['combined_text'].values,
    df['label'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 4. Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # lightweight
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=256)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:

# 5. Dataset class
class InternSelectionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = InternSelectionDataset(train_encodings, train_labels)
val_dataset = InternSelectionDataset(val_encodings, val_labels)

In [21]:
# 6. Model setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    hidden_dropout_prob=0.3
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=2e-5, weight_decay=0.02)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 7. Scheduler
num_epochs = 12
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
patience = 3
min_delta = 1e-4
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_postfix(train_loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    val_loop = tqdm(val_loader, desc=f"Epoch {epoch + 1} Validation", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            labels = batch["labels"]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            val_loop.set_postfix(val_loss=val_loss/len(all_preds), val_acc=accuracy_score(all_labels, all_preds))

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)

    print(f"✅ Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

    if avg_val_loss < best_val_loss - min_delta:
        best_val_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered. No improvement for {patience} epochs.")
        break



Epoch 1 ✅ Train Loss: 1.1280 | Val Loss: 1.0667 | Val Acc: 0.4068


Epoch 2 ✅ Train Loss: 0.9620 | Val Loss: 0.8603 | Val Acc: 0.6294


Epoch 3 ✅ Train Loss: 0.7878 | Val Loss: 0.8443 | Val Acc: 0.6632


Epoch 4 ✅ Train Loss: 0.6240 | Val Loss: 0.6672 | Val Acc: 0.7564


Epoch 5 ✅ Train Loss: 0.5646 | Val Loss: 0.5357 | Val Acc: 0.7832


...

Epoch 7 ✅ Train Loss: 0.4916 | Val Loss: 0.4677 | Val Acc: 0.8205


Epoch 8 ✅ Train Loss: 0.4700 | Val Loss: 0.4650 | Val Acc: 0.8252


Epoch 9 ✅ Train Loss: 0.4428 | Val Loss: 0.5032 | Val Acc: 0.8170


Epoch 10 ✅ Train Loss: 0.4377 | Val Loss: 0.4866 | Val Acc: 0.8193


Epoch 11 ✅ Train Loss: 0.4155 | Val Loss: 0.4930 | Val Acc: 0.8228
Early stopping triggered. Validation loss has not improved for 3 epochs.


In [ ]:
print(classification_report(all_labels, all_preds, target_names=["no", "yes"]))


In [ ]:
from sklearn.metrics import classification_report

model.eval()
preds, labels = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        labels.extend(batch["labels"].cpu().numpy())

label_map = {0: "hate_speech", 1: "offensive_language", 2: "neither"}
print(classification_report(labels, preds, target_names=label_map.values()))


                    precision    recall  f1-score   support

       hate_speech       0.75      0.79      0.77       286
offensive_language       0.82      0.80      0.81       286
           neither       0.91      0.87      0.89       286

          accuracy                           0.82       858
         macro avg       0.83      0.82      0.82       858
      weighted avg       0.83      0.82      0.82       858



In [ ]:
# Save the model
torch.save(model.state_dict(), "best_model_bert.pt")
print("Model saved as best_model_bert.pt")


Model saved as best_model_bert.pt


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=3)
model.load_state_dict(torch.load("best_model_bert.pt"))
model.to(device)
model.eval()  # Set the model to evaluation mode

print("Model loaded from best_model_bert.pt")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: 'best_model_bert.pt'

In [ ]:
from transformers import AutoTokenizer
import torch

# Initialize tokenizer for BERT
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

label_map = {
    0: "Hate",
    1: "Offensive",
    2: "Neither"
}

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)  # Get probabilities from logits
        predicted_class = torch.argmax(probs, dim=1).item()  # Get the predicted class (index)
    return label_map[predicted_class], probs.cpu().numpy()

# Example texts to predict
texts = [
    "I love you but your actions disgust me!",  # Mixed emotion (hate + love)
    "You are so stupid and annoying!",  # Offensive language
    "I can't believe how much I adore you, you're perfect.",  # Positive sentiment
    "This is just a simple sentence.",  # Neutral statement
    "You are a piece of shit!",  # Hate speech
    "I appreciate your help, it's been great!",  # Positive sentiment
    "Honestly, I don't even care anymore. Just leave me alone.",  # Negative tone, indifferent
    "I just want the best for everyone, peace and love.",  # Positive, neutral
    "This is the worst thing I've ever seen.",  # Strong negative emotion (offensive)
    "I would die for you, my love.",  # Extreme positive emotion
    "Why can't you just be normal? Ugh!",  # Annoyance + frustration (offensive)
    "Is it really so hard to be kind? Why do people act so heartless?",  # Disbelief mixed with frustration
    "I adore the fact that you don't care about what anyone thinks.",  # Positive with some sarcasm
    "We need to stop spreading hate and negativity in the world.",  # Neutral but a bit political
    "Your stupid jokes make my day. Please stop.",  # Mixed sentiment (offensive + playful)
    "Please, let's keep this civil and respectful.",  # Neutral with a respectful tone
    "I'm not angry, just disappointed.",  # Mixed negative sentiment
    "Love is patient, love is kind.",  # Positive (romantic)
    "You're the worst thing that has ever happened to me.",  # Hate speech
    "You're so annoying, I can't stand it.",  # Offensive language
]

# Loop through each text and make predictions
for text in texts:
    pred_class, probabilities = predict(text)
    print(f"Text: {text}\nPredicted Class: {pred_class}, Probabilities: {probabilities}\n")


Text: I love you but your actions disgust me!
Predicted Class: Neither, Probabilities: [[0.35511875 0.18668386 0.45819733]]

Text: You are so stupid and annoying!
Predicted Class: Hate, Probabilities: [[0.54450387 0.35307902 0.10241712]]

Text: I can't believe how much I adore you, you're perfect.
Predicted Class: Neither, Probabilities: [[0.06298984 0.06506774 0.8719424 ]]

Text: This is just a simple sentence.
Predicted Class: Neither, Probabilities: [[0.3052921  0.14032164 0.5543862 ]]

Text: You are a piece of shit!
Predicted Class: Hate, Probabilities: [[0.5163689 0.4784821 0.005149 ]]

Text: I appreciate your help, it's been great!
Predicted Class: Neither, Probabilities: [[0.06302823 0.04421059 0.8927612 ]]

Text: Honestly, I don't even care anymore. Just leave me alone.
Predicted Class: Neither, Probabilities: [[0.2561737  0.11162211 0.6322041 ]]

Text: I just want the best for everyone, peace and love.
Predicted Class: Neither, Probabilities: [[0.06031108 0.02881691 0.91087204

## Roberta Large

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from transformers import RobertaTokenizer
from torch.utils.data import Dataset, DataLoader

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=256)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [ ]:
class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)

In [ ]:
from transformers import RobertaForSequenceClassification, get_scheduler
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import torch
import numpy as np

# Load model and move to device
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3, hidden_dropout_prob=0.2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define optimizer with weight decay (L2 regularization)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.02)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Scheduler for learning rate decay (linear warmup followed by linear decay)
num_epochs = 12
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
)

# Early stopping parameters
patience = 2  # Number of epochs to wait for improvement in validation loss
best_val_loss = float("inf")  # Initialize best validation loss
epochs_without_improvement = 0  # Counter for patience


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

for epoch in range(num_epochs):
    # Train Phase
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        loop.set_postfix(train_loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)

    # ----- Validation Phase with Live Updates -----
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    val_loop = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} Validation", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            labels = batch["labels"]

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Live update of validation metrics in the tqdm bar
            val_accuracy = accuracy_score(all_labels, all_preds)
            val_loop.set_postfix(val_loss=val_loss/len(all_preds), val_acc=val_accuracy)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss  # Update the best validation loss
        epochs_without_improvement = 0  # Reset the counter
    else:
        epochs_without_improvement += 1  # Increment the counter

    # If the patience is exceeded, stop training
    if epochs_without_improvement >= patience:
        print(f"Early stopping triggered at epoch {epoch + 1}")
        break

    print(f"Epoch {epoch + 1} ✅ Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

Epoch 1 ✅ Train Loss: 1.1065 | Val Loss: 1.0932 | Val Acc: 0.5089


Epoch 2 ✅ Train Loss: 1.0707 | Val Loss: 0.7883 | Val Acc: 0.6612


Epoch 3 ✅ Train Loss: 0.7366 | Val Loss: 0.4961 | Val Acc: 0.8026


Epoch 4 ✅ Train Loss: 0.5447 | Val Loss: 0.4819 | Val Acc: 0.8159


Epoch 5 ✅ Train Loss: 0.4948 | Val Loss: 0.4827 | Val Acc: 0.8096


Epoch 6 ✅ Train Loss: 0.4828 | Val Loss: 0.4662 | Val Acc: 0.8143


Epoch 7 ✅ Train Loss: 0.4478 | Val Loss: 0.4566 | Val Acc: 0.8228


Epoch 8 ✅ Train Loss: 0.4181 | Val Loss: 0.4587 | Val Acc: 0.8291


Early stopping triggered at epoch 9


In [ ]:
from sklearn.metrics import classification_report

model.eval()
preds, labels = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        labels.extend(batch["labels"].cpu().numpy())

label_map = {0: "hate_speech", 1: "offensive_language", 2: "neither"}
print("\nClassification Report on Validation Set:")
print(classification_report(labels, preds, target_names=label_map.values()))



Classification Report on Validation Set:
                    precision    recall  f1-score   support

       hate_speech       0.75      0.81      0.78       429
offensive_language       0.84      0.77      0.80       429
           neither       0.90      0.90      0.90       429

          accuracy                           0.83      1287
         macro avg       0.83      0.83      0.83      1287
      weighted avg       0.83      0.83      0.83      1287



In [ ]:
# Save the model
torch.save(model.state_dict(), "best_model.pt")
print("Model saved as best_model.pt")


Model saved as best_model.pt


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the same model architecture
model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=3)
model.load_state_dict(torch.load("best_model.pt", map_location=device))
model.to(device)
model.eval()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [ ]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

label_map = {
    0: "Hate",
    1: "Offensive",
    2: "Neither"
}

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
    return label_map[predicted_class], probs.cpu().numpy()

text = "Fuck in Love."
pred_class, probabilities = predict(text)
print(f"Predicted Class: {pred_class}, Probabilities: {probabilities}")


Predicted Class: Offensive, Probabilities: [[0.1366126  0.8605211  0.00286638]]
